In [29]:
import os
import sys
import pyproj
import geopandas as gpd
from shapely.geometry import mapping
import rasterio
from rasterio.mask import mask
import numpy as np
from shapely.geometry import box
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import cKDTree

In [30]:
from utils import load_geotif, save_features_to_shapefile

In [31]:
conda_prefix = sys.prefix
proj_lib = os.path.join(conda_prefix, "Library", "share", "proj")
pyproj.datadir.set_data_dir(proj_lib)
print(f"Manually set PROJ_LIB to: {pyproj.datadir.get_data_dir()}")

Manually set PROJ_LIB to: c:\Users\david.bruner\.conda\envs\geoaienv\Library\share\proj


In [32]:
dataset, geotransform, projection = load_geotif('image_data/auckland-0075m-urban-aerial-photos-2017.tif')

c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:4216: RuntimeWarning: PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\13\share\contrib\postgis-3.4\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.
  return _gdal.Dataset_GetProjection(self, *args)
c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:4216: RuntimeWarning: PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\13\share\contrib\postgis-3.4\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.
  return _gdal.Dataset_GetProjection(self, *args)
c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:4216: RuntimeWarning: The definition of projected CRS EPSG:2193 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operati

In [33]:
def load_bounding_boxes(file_path):
    return gpd.read_file(file_path)

def find_close_boxes(gdf1: gpd.GeoDataFrame, gdf2: gpd.GeoDataFrame, distance_threshold: float = 1.0) -> gpd.GeoDataFrame:
    """
    Find close pairs of bounding boxes between two GeoDataFrames using their centroids.

    :param gdf1: GeoDataFrame containing bounding boxes from the first image
    :param gdf2: GeoDataFrame containing bounding boxes from the second image
    :param distance_threshold: Maximum distance to consider centroids as matching
    :return: GeoDataFrame containing matched pairs of bounding boxes
    """
    # Calculate centroids
    gdf1['centroid'] = gdf1.geometry.centroid
    gdf2['centroid'] = gdf2.geometry.centroid

    # Convert centroids to numpy arrays for KDTree
    centroids1_array = np.array([(c.x, c.y) for c in gdf1['centroid']])
    centroids2_array = np.array([(c.x, c.y) for c in gdf2['centroid']])

    # Build KDTree for faster nearest neighbor search
    tree = cKDTree(centroids2_array)

    # Find nearest neighbors
    distances, indices = tree.query(centroids1_array, k=1, distance_upper_bound=distance_threshold)

    # Create a list of matched pairs
    matched_pairs = [(i, j) for i, (d, j) in enumerate(zip(distances, indices)) if d <= distance_threshold]

    # Create a new GeoDataFrame with matched pairs
    if matched_pairs:
        matched_df = gpd.GeoDataFrame({
            'index_1': [pair[0] for pair in matched_pairs],
            'index_2': [pair[1] for pair in matched_pairs],
            'geometry_1': gdf1.loc[[pair[0] for pair in matched_pairs], 'geometry'].values,
            'geometry_2': gdf2.loc[[pair[1] for pair in matched_pairs], 'geometry'].values,
            'centroid_1': gdf1.loc[[pair[0] for pair in matched_pairs], 'centroid'].values,
            'centroid_2': gdf2.loc[[pair[1] for pair in matched_pairs], 'centroid'].values,
            'distance': [distances[pair[0]] for pair in matched_pairs]
        })

        # Set the geometry to the centroid of the first image for spatial operations
        matched_df.set_geometry('centroid_1', inplace=True)

        # Set the CRS to match the input GeoDataFrames
        matched_df.set_crs(gdf1.crs, inplace=True)
    else:
        # If no matches found, return an empty GeoDataFrame with the correct structure
        matched_df = gpd.GeoDataFrame(columns=['index_1', 'index_2', 'geometry_1', 'geometry_2', 'centroid_1', 'centroid_2', 'distance'], geometry='centroid_1', crs=gdf1.crs)

    return matched_df

In [34]:
BOXES_PATH_2016 = 'C:/Users/david.bruner/Documents/car_bboxes_akl_whl_2016.gpkg'
BOXES_PATH_2017 = 'C:/Users/david.bruner/Documents/car_bboxes_akl_whl_2017.gpkg'

boxes1 = load_bounding_boxes(BOXES_PATH_2016)
boxes2 = load_bounding_boxes(BOXES_PATH_2017)

close_pairs = find_close_boxes(boxes1, boxes2)

In [35]:
close_pairs_features_1 = close_pairs['geometry_1'].tolist()
close_pairs_features_2 = close_pairs['geometry_2'].tolist()
print(close_pairs_features_1)

[<MULTIPOLYGON (((1752860.115 5933785.485, 1752860.115 5933780.305, 1752864.5...>, <MULTIPOLYGON (((1752785.915 5933863.045, 1752785.915 5933858.565, 1752790.6...>, <MULTIPOLYGON (((1752961.335 5933852.685, 1752961.335 5933848.485, 1752966.3...>, <MULTIPOLYGON (((1752843.175 5933730.115, 1752843.175 5933725.635, 1752847.8...>, <MULTIPOLYGON (((1752800.685 5933716.815, 1752800.685 5933711.775, 1752804.8...>, <MULTIPOLYGON (((1752980.585 5933739.215, 1752980.585 5933734.175, 1752984.6...>, <MULTIPOLYGON (((1752970.645 5933845.335, 1752970.645 5933841.415, 1752975.8...>, <MULTIPOLYGON (((1752795.155 5933705.125, 1752795.155 5933700.225, 1752799.2...>, <MULTIPOLYGON (((1752953.355 5933814.885, 1752953.355 5933810.895, 1752958.3...>, <MULTIPOLYGON (((1752789.275 5933709.885, 1752789.275 5933704.845, 1752793.2...>, <MULTIPOLYGON (((1752799.635 5933694.625, 1752799.635 5933689.515, 1752803.5...>, <MULTIPOLYGON (((1752792.985 5933706.665, 1752792.985 5933701.905, 1752797.1...>, <MULTIPOLYGON (

In [36]:
save_features_to_shapefile(close_pairs_features_1, projection, 'image_data/close_pairs_2016')
save_features_to_shapefile(close_pairs_features_2, projection, 'image_data/close_pairs_2017')

GeoDataFrame saved to image_data/close_pairs_2016
GeoDataFrame saved to image_data/close_pairs_2017
